# This is a Direct Copy from Nick Wan's Benchmark Solution

In [6]:
import os
import pandas as pd
import numpy as np
import catboost as cb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
for dirname, _, filenames in os.walk(''):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [7]:
df = pd.read_csv('train.csv')

feats = [
    'release_speed', 'release_pos_x', 'release_pos_z', 'balls',
    'strikes', 'game_year', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z',
    'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'vx0', 'vy0',
    'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'effective_speed',
    'release_spin_rate', 'release_extension', 'release_pos_y',
    'pitch_number', 'spin_axis', 'is_lhp', 'is_lhb', 'spray_angle'
]

target = 'outcome_code'

In [9]:
folds = 3
kf = KFold(folds, shuffle=True)
outputs = pd.DataFrame()

for train_idx, test_idx in kf.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[test_idx]
    model = cb.CatBoostClassifier(iterations=1000, verbose=False, loss_function='MultiClassOneVsAll', eval_metric='AUC')
    model.fit(train.loc[:, feats], train[target])
    _df = pd.DataFrame(model.predict_proba(test.loc[:, feats]), index=test.index)
    outputs = pd.concat([outputs, _df])


df = pd.concat([df, outputs], axis=1)
df['e'] = df.loc[:, range(5)].sum(axis=1).sub(1)
df[0] = df[0].sub(df['e'])

In [10]:
df = df.rename(columns={0:'out',1:'single',2:'double',3:'triple',4:'home_run'})
df.loc[:, ['uid','out','single','double','triple','home_run']].sample(10)

,uid,out,single,double,triple,home_run
6123,8665,0.784698,0.176292,0.027150,0.004130,0.007731
34815,50952,0.678336,0.237613,0.050845,0.009978,0.023228
48771,70957,0.611803,0.213306,0.050672,0.006388,0.117831
24241,35167,0.650873,0.175320,0.054567,0.004294,0.114946
4439,6271,0.543482,0.252302,0.082158,0.016766,0.105293
4185,5917,0.766780,0.169683,0.036527,0.004888,0.022121
20454,29699,0.782824,0.160004,0.037997,0.002828,0.016347
6889,9953,0.668887,0.164266,0.065686,0.004118,0.097042
15185,21970,0.649563,0.230417,0.052857,0.014072,0.053091
3945,5583,0.666012,0.274355,0.039358,0.008090,0.012185


In [11]:
roc_auc_score(df[target], df.loc[:, ['out','single','double','triple','home_run']], multi_class='ovr')

0.6945824975826527